In [59]:
!pip install preprocessor
!pip install vaderSentiment
!pip install torchmetrics
!pip install sentence-transformers


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [90]:
import pandas as pd
import json
import os
import preprocessor as p
import re
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

from sentence_transformers import SentenceTransformer

In [31]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vincentdandenault/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vincentdandenault/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
dir_path = 'Data'
censored_path = 'censored_tweets'
control_path = 'plusone_control_tweets'

In [33]:
dfs = []
for r, d, f in os.walk('Data'):
    for file in f:
        if 'withheldtweets.json' in file or 'plus_one_control.json' in file:  # alt: if ‘control' in file:
            dfs.append(pd.read_json('%s/%s' % (r, file), lines=True))

df_cen = pd.concat(dfs)

In [34]:
df_cen.head(10)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,filter_level,lang,timestamp_ms,linked,display_text_range,withheld_in_countries,extended_entities,possibly_sensitive,retweeted_status,withheld_copyright
0,2021-02-26 10:54:27+00:00,1365253891839971330,1365253891839971328,"CHINE - Depuis le 1er janvier 2021, une loi pe...","<a href=""https://mobile.twitter.com"" rel=""nofo...",True,NaN,NaN,NaN,NaN,...,low,fr,2021-02-26 10:54:27.662,quoted,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-02-26 10:48:35+00:00,1365252415444946945,1365252415444946944,#Balakot \nPak Army is our pride ❤️👍 https://t...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,en,2021-02-26 10:48:35.662,no,"[0, 35]",[IN],"{'media': [{'id': 1365252409015033857, 'id_str...",0.0,NaN,NaN
2,2021-02-26 10:45:51+00:00,1365251727574900738,1365251727574900736,RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:45:51.661,no,NaN,[IN],NaN,NaN,{'created_at': 'Thu Feb 25 18:59:12 +0000 2021...,NaN
3,2021-02-26 10:45:51+00:00,1365251727574900738,1365251727574900736,RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:45:51.661,retweeted,NaN,[IN],NaN,NaN,{'created_at': 'Thu Feb 25 18:59:12 +0000 2021...,NaN
4,2021-02-26 10:38:57+00:00,1365249991137251328,1365249991137251328,RT @SaniaNishtar: سید ابراہیم کا تعلق قبائلی ض...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:38:57.662,no,NaN,[IN],NaN,NaN,{'created_at': 'Fri Feb 26 05:23:00 +0000 2021...,NaN
5,2021-02-26 10:10:45+00:00,1365242894362279938,1365242894362279936,RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,low,ar,2021-02-26 10:10:45.659,no,NaN,[IL],NaN,NaN,{'created_at': 'Thu Feb 25 19:04:40 +0000 2021...,NaN
6,2021-02-26 10:10:45+00:00,1365242894362279938,1365242894362279936,RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,low,ar,2021-02-26 10:10:45.659,retweeted,NaN,[IL],NaN,NaN,{'created_at': 'Thu Feb 25 19:04:40 +0000 2021...,NaN
7,2021-02-26 10:10:46+00:00,1365242898569134080,1365242898569134080,RT @Saimhun: Mujhe aj bhi batting ki bari ni d...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,hi,2021-02-26 10:10:46.662,no,NaN,[IN],"{'media': [{'id': 1365238103892516864, 'id_str...",0.0,{'created_at': 'Fri Feb 26 09:51:44 +0000 2021...,NaN
8,2021-02-26 10:10:46+00:00,1365242898569134080,1365242898569134080,RT @Saimhun: Mujhe aj bhi batting ki bari ni d...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,hi,2021-02-26 10:10:46.662,retweeted,NaN,[IN],"{'media': [{'id': 1365238103892516864, 'id_str...",0.0,{'created_at': 'Fri Feb 26 09:51:44 +0000 2021...,NaN
9,2021-02-26 10:59:24+00:00,1365255137552457730,1365255137552457728,RT @iVeenaKhan: (اے پیغمبرﷺ! لوگوں سے) کہہ دو ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:59:24.663,no,NaN,[IN],NaN,NaN,{'created_at': 'Fri Feb 26 05:42:25 +0000 2021...,NaN


## Preprocessing

In [35]:
features_to_keep = ['id','text', 'truncated', 'user', 'user-location', 'user-verified', 'user-followers_count', \
                    'withheld_in_countries', 'entities', 'lang', 'extended_entities', \
                   'possibly_sensitive', 'extended_tweet']

In [36]:
worthKeeping = ["text", "truncated", "user",
                "withheld_in_countries", "entities", "lang",
                "possibly_sensitive", "extended_tweet"]
df_cen = df_cen[worthKeeping]

In [37]:
df_cen['possibly_sensitive'] = df_cen['possibly_sensitive'].fillna(0.0)

In [38]:
dfRaw = df_cen.values
for line in dfRaw:
    #if not pd.isna(line[-1]):
        #line[0] = line[-1]["full_text"]
        
    #remove urls from tweets
    #they are shortened anyway so we can't make use of them
    line[0] = re.sub(r'http\S+', '', str(line[0]))
    
    #flatten retweets
    line[0] = re.sub(r'RT @\S+:', '', str(line[0]))

dfRaw = np.delete(dfRaw, len(worthKeeping)-1, axis=1) #remove "extended_tweet"
worthKeeping.remove("extended_tweet")

dfRaw = np.delete(dfRaw, 1, axis=1) #remove "truncated"
worthKeeping.remove("truncated")

In [39]:
for line in dfRaw:
    line[3] = [x["text"] for x in line[3]["hashtags"]]
worthKeeping[3] = "hashtags"

In [40]:
#create a feature for user-verified and user-followers_count
verified = [line[1]["verified"] for line in dfRaw]
followers = [line[1]["followers_count"] for line in dfRaw]
user_id = [line[1]["id"] for line in dfRaw]

#for the location, Rebekah suggested to only spot the country name and discard the rest
listOfCountries = ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe']
def findCountry(x):
    for country in listOfCountries:
        if x and country in x:
            return country
    return None

location = [findCountry(line[1]["location"]) for line in dfRaw]

dfRaw = np.c_[dfRaw, verified, followers, location, user_id]
worthKeeping += ["verified_account", "followers_count", "location", "user_id"]

In [41]:
withheld = []
for line in dfRaw:
    if not isinstance(line[2], list):
        line[2] = []
    withheld.append(len(line[2]) != 0)
        
dfRaw = np.c_[dfRaw, withheld]
worthKeeping += ["withheld_anywhere"]

In [42]:
sentiment = SentimentIntensityAnalyzer()
#we made the assumption that sentiment analysis for this analyzer only works for english
res = np.array([[x for x in sentiment.polarity_scores(line[0]).values()] if line[4] == "en" else [0.0, 0.0, 0.0, 0.0] for line in dfRaw])

dfRaw = np.c_[dfRaw, res]
worthKeeping += ["neg", "neu", "pos", "compound"]


In [43]:
#popularity feature:
#build a score based on the values of followers_count, favourites_count, statuses_count
#compute a score from 0 to 1 for each, with (x - min)/(max - min), then comptute the average of these scores 

followers_count = np.array([line[1]["followers_count"] for line in dfRaw])
favourites_count = np.array([line[1]["favourites_count"] for line in dfRaw])
statuses_count = np.array([line[1]["statuses_count"] for line in dfRaw])

def normalize(array):
    return (array - np.min(array)) / (np.max(array) - np.min(array))

score = (1/3) * (normalize(followers_count) + normalize(favourites_count) + normalize(statuses_count))
dfRaw = np.c_[dfRaw, score]
worthKeeping += ["popularity_score"]


In [44]:
#reassemble the data in a pandas dataframe
df_cen = pd.DataFrame(dfRaw, columns = worthKeeping)
df_cen

,text,user,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score
0,"CHINE - Depuis le 1er janvier 2021, une loi pe...","{'id': 1184106601122156544, 'id_str': '1184106...",[],[],fr,0.0,False,529,None,1184106601122156544,False,0.0,0.0,0.0,0.0,0.000075
1,#Balakot \nPak Army is our pride ❤️👍,"{'id': 966615015716458497, 'id_str': '96661501...",[IN],[Balakot],en,0.0,False,93,None,966615015716458497,True,0.0,0.789,0.211,0.34,0.000542
2,لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ \nہم...,"{'id': 407984569, 'id_str': '407984569', 'name...",[IN],[],ur,0.0,False,753,Pakistan,407984569,True,0.0,0.0,0.0,0.0,0.007183
3,لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ \nہم...,"{'id': 407984569, 'id_str': '407984569', 'name...",[IN],[],ur,0.0,False,753,Pakistan,407984569,True,0.0,0.0,0.0,0.0,0.007183
4,سید ابراہیم کا تعلق قبائلی ضلع کرم سے ہے۔ احس...,"{'id': 1022545022447759360, 'id_str': '1022545...",[IN],[],ur,0.0,False,3526,None,1022545022447759360,True,0.0,0.0,0.0,0.0,0.020478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70883,ถ้าได้รักใครสักคนขึ้นมาจริงๆ ฉันคงโอบกอดทุกข้...,"{'id': 733700890129883136, 'id_str': '73370089...",[],[],th,0.0,False,110,None,733700890129883136,False,0.0,0.0,0.0,0.0,0.002502
70884,リンクの妖精 ヴィクトル・ニキフォロフ＆\nお花畑のシンデレラ クリストフ・ジャコメッティ...,"{'id': 827750070724354048, 'id_str': '82775007...",[],"[クリストフ・ジャコメッティ生誕祭2021, クリストフ・ジャコメッティ誕生祭2021, ク...",ja,0.0,False,29,None,827750070724354048,False,0.0,0.0,0.0,0.0,0.012251
70885,,"{'id': 1352109785383116805, 'id_str': '1352109...",[],[],und,0.0,False,28,None,1352109785383116805,False,0.0,0.0,0.0,0.0,0.000358
70886,AĞLIYORUM ÖĞLE ARAMIZI 40 DAKŞKA YAPMIŞLAR AĞL...,"{'id': 1322266379203022848, 'id_str': '1322266...",[],[],tr,0.0,False,81,None,1322266379203022848,False,0.0,0.0,0.0,0.0,0.000347


In [45]:
cleanCols = filter(lambda x: x != "user", worthKeeping)
df_clean = df_cen[cleanCols]

In [46]:
p.set_options(p.OPT.URL, p.OPT.NUMBER, p.OPT.URL, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.HASHTAG)
df_processed = df_clean.copy()
df_processed["text"] = df_clean.apply({"text": lambda line: p.clean(line)}) 

In [47]:
df_noduplicates = df_processed.copy()
df_noduplicates['withheld_in_countries'] = df_processed['withheld_in_countries'].astype(str)
df_noduplicates['hashtags'] = df_processed['hashtags'].astype(str)

df_noduplicates.drop_duplicates()
cleanCols = filter(lambda x: x != "user", worthKeeping)
df_clean = df_noduplicates[cleanCols]

In [48]:
#df_english = df_noduplicates[df_noduplicates['lang'] == "en"] 
#df_turkish = df_noduplicates[df_noduplicates['lang'] == "tr"] 
#df_urdu = df_noduplicates[df_noduplicates['lang'] == "ur"]
#df_japanese = df_noduplicates[df_noduplicates['lang'] == "ja"] 
#df_spanish = df_noduplicates[df_noduplicates['lang'] == "es"] 
#df_thai = df_noduplicates[df_noduplicates['lang'] == "th"] 
#df_portuguese = df_noduplicates[df_noduplicates['lang'] == "pt"] 
#df_arabic = df_noduplicates[df_noduplicates['lang'] == "ar"] 
#df_indian = df_noduplicates[df_noduplicates['lang'] == "in"] 

In [49]:
df_english = df_clean[df_noduplicates['lang'] == "en"] 
print(df_english.shape)

(25830, 15)


In [50]:
TO_CSV = False
if TO_CSV:
    df_english.to_csv('df_english_clean', encoding='utf-8', index=False)

## Feature Encoding

In [34]:
#vectorizer = CountVectorizer(stop_words='english')
#vectorizer.fit(flat_list_text_tokenized)
#def bow_tranform_wrapper(sentences): 
#    if isinstance(sentences, list): 
#        return vectorizer.transform(sentences)
#    else:
#        print('Not a list! Ingore message: ' + str(sentences))
#        return vectorizer.transform([sentences])

CountVectorizer(stop_words='english')

In [51]:
df_english["withheld_anywhere"] = df_english["withheld_anywhere"].astype(int)
y = df_english["withheld_anywhere"]

/var/folders/_m/v82wb7j964v3b7b70x4rrvvr0000gn/T/ipykernel_33989/2991236561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english["withheld_anywhere"] = df_english["withheld_anywhere"].astype(int)


In [61]:
stop_words = set(stopwords.words('english'))
def word_tokenize_wrapper(sentence): 
    if isinstance(sentence, str): 
        sentence = " ".join(sentences)
        return nltk.sent_tokenize(sentence)
    else:
        print('Not a string! Ingore message: ' + str(sentence))
        return ''

In [117]:
country_label = preprocessing.LabelEncoder()
countries_encoded = country_label.fit_transform(list(df_english.location.values))
df_english['Country_encoded'] = countries_encoded

/var/folders/_m/v82wb7j964v3b7b70x4rrvvr0000gn/T/ipykernel_33989/687327608.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['Country_encoded'] = countries_encoded


In [118]:
df_english['Country_encoded'].values

array([121, 121, 116, ..., 121, 121, 121])

In [119]:
df_english.dtypes

text                      object
withheld_in_countries     object
hashtags                  object
lang                      object
possibly_sensitive       float64
verified_account            bool
followers_count           object
location                  object
user_id                   object
withheld_anywhere          int64
neg                       object
neu                       object
pos                       object
compound                  object
popularity_score          object
text_tokenized_list       object
Country_encoded            int64
dtype: object

In [62]:
text_tokenized_list = list(df_english.apply(lambda x: word_tokenize_wrapper(x['text']), axis=1))

In [128]:
df_english.dtypes

text                      object
withheld_in_countries     object
hashtags                  object
lang                      object
possibly_sensitive       float64
verified_account         float64
followers_count            int64
location                  object
user_id                    int64
withheld_anywhere          int64
neg                      float64
neu                      float64
pos                      float64
compound                 float64
popularity_score         float64
text_tokenized_list       object
Country_encoded            int64
dtype: object

In [127]:
df_english = df_english.astype({"possibly_sensitive": float, "verified_account": float,\
                                'followers_count':int, 'user_id': int, 'neg': float, 'neu': float, 
                               'pos': float, 'compound': float, 'popularity_score': float})

In [129]:
df_english['text_tokenized_list'] = text_tokenized_list
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = sbert_model.encode(text_tokenized_list)
df_english['text_embeddings'] = sentence_embeddings

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
features_encoded = ['text_embeddings', 'possibly_sensitive', 'verified_account', 'Country_encoded ',\
                    'followers_count', 'user_id', 'neg', 'neu', 'pos', 'compound', 'popularity_score']

In [ ]:
X = df_english[features_encoded].copy().to_numpy()

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'cleaned_sentences' is not defined

In [40]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_train.shape)

(20664,)
(5166,)
(20664,)
(20664,)


In [41]:
seed = 42
svm = SVC(random_state=seed)
random_forest = RandomForestClassifier(random_state=seed)
gnb = GaussianNB()
models = {'SVM': svm, 'Random Forest': random_forest, 'Gaussian Naive Bayes': gnb}
scores = {}
for name in models.keys(): 
    y_pred = models[name].fit(X_train, y_train).predict(X_test)
    scores[name] = confusion_matrix(y_true, y_pred)

ValueError: setting an array element with a sequence.